## A Movie Recommender System

- The objective is to create a recommender system to find out what animation movies to watch next after seeing Antz.

## Import Libraries

In [1]:
#code block 1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#allows for up to 500 columns to be displayed when viewing a dataframe
pd.set_option('display.max_columns',500) 

import warnings
warnings.filterwarnings("ignore")

#sets style to seaborn-colorblind
plt.style.use('seaborn-colorblind')

#graphs automatically show without plt.show()
%matplotlib inline

## Import Data

In [2]:
#code block 2
#does not set the first column as index
#sets the first row as header

df_movies = pd.read_csv("data/movies.csv", index_col=None, header=0)

df_genres = pd.read_csv("data/movies_genres.csv", index_col=None, header=0)

df_ratings = pd.read_csv("data/ratings_small.csv", index_col=0, header=0)

In [3]:
#code block 3
#shows the properties of df_movies and its top rows

display(df_movies.info())
df_movies.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    45463 non-null  int64  
 1   imdb_id               45446 non-null  object 
 2   adult                 45463 non-null  bool   
 3   budget                45463 non-null  int64  
 4   original_language     45452 non-null  object 
 5   original_title        45463 non-null  object 
 6   overview              44509 non-null  object 
 7   popularity            45460 non-null  float64
 8   production_countries  45460 non-null  object 
 9   release_date          45376 non-null  object 
 10  revenue               45460 non-null  float64
 11  runtime               45203 non-null  float64
 12  tagline               20412 non-null  object 
 13  title                 45460 non-null  object 
 14  video                 45460 non-null  object 
 15  vote_average       

None

,id,imdb_id,adult,budget,original_language,original_title,overview,popularity,production_countries,release_date,revenue,runtime,tagline,title,video,vote_average,vote_count
0,862,tt0114709,False,30000000,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",10/30/95,373554033.0,81.0,NaN,Toy Story,False,7.7,5415.0
1,8844,tt0113497,False,65000000,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",12/15/95,262797249.0,104.0,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,15602,tt0113228,False,0,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,"[{'iso_3166_1': 'US', 'name': 'United States o...",12/22/95,0.0,101.0,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,31357,tt0114885,False,16000000,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,"[{'iso_3166_1': 'US', 'name': 'United States o...",12/22/95,81452156.0,127.0,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,11862,tt0113041,False,0,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,"[{'iso_3166_1': 'US', 'name': 'United States o...",2/10/95,76578911.0,106.0,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
#code block 4
#shows the properties of df_genres and its top rows

display(df_genres.info())
df_genres.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43021 entries, 0 to 43020
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      43021 non-null  int64 
 1   Genre1  43021 non-null  object
 2   Genre2  28462 non-null  object
 3   Genre3  13982 non-null  object
 4   Genre4  4397 non-null   object
 5   Genre5  1018 non-null   object
 6   Genre6  184 non-null    object
 7   Genre7  27 non-null     object
 8   Genre8  3 non-null      object
dtypes: int64(1), object(8)
memory usage: 3.0+ MB


None

,id,Genre1,Genre2,Genre3,Genre4,Genre5,Genre6,Genre7,Genre8
0,2,Drama,Crime,NaN,NaN,NaN,NaN,NaN,NaN
1,3,Drama,Comedy,NaN,NaN,NaN,NaN,NaN,NaN
2,5,Crime,Comedy,NaN,NaN,NaN,NaN,NaN,NaN
3,6,Action,Thriller,Crime,NaN,NaN,NaN,NaN,NaN
4,11,Adventure,Action,ScienceFiction,NaN,NaN,NaN,NaN,NaN


In [5]:
#code block 5
#shows the properties of df_ratings and its top rows

display(df_ratings.info())
df_ratings.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3247720 entries, 0 to 3247719
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 123.9 MB


None

,userId,movieId,rating,timestamp
0,1923,862,3.0,858335006
1,2103,862,5.0,946044912
2,5380,862,1.0,878941641
3,6177,862,4.0,859415226
4,6525,862,4.0,857388995


## Data preparation to filter based on animation 

- To filter the movies based on it being an animation

### Create df_genres_pivot

In [6]:
#code block 6
#creates a dataframe by melting df_genres

df_genres_melt = pd.melt(df_genres, id_vars=['id'],
                         value_vars=['Genre1','Genre2','Genre3','Genre4','Genre5','Genre6','Genre7','Genre8'])

In [7]:
#code block 7

df_genres_melt.head()

,id,variable,value
0,2,Genre1,Drama
1,3,Genre1,Drama
2,5,Genre1,Crime
3,6,Genre1,Action
4,11,Genre1,Adventure


In [8]:
#code block 8
#creates a new column which is set to 1 in df_genres_melt

df_genres_melt['count'] = 1

In [9]:
#code block 9
#views top 5 rows in df_genres_melt

df_genres_melt.head()

,id,variable,value,count
0,2,Genre1,Drama,1
1,3,Genre1,Drama,1
2,5,Genre1,Crime,1
3,6,Genre1,Action,1
4,11,Genre1,Adventure,1


In [10]:
#code block 10
#converts df_genres_melt into a pivot table

df_genres_pivot = pd.pivot_table(df_genres_melt, index='id', columns='value', values='count')

In [11]:
#code block 11
#views top 5 rows in df_genres_pivot

df_genres_pivot = df_genres_pivot.reset_index()
df_genres_pivot.head()

value,id,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,ScienceFiction,TVMovie,Thriller,War,Western
0,2,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,11,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [12]:
#code block 12
#fills the null values with 0

df_genres_pivot = df_genres_pivot.fillna(0)
df_genres_pivot.head()

value,id,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,ScienceFiction,TVMovie,Thriller,War,Western
0,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,11,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### Filter df_movies based on animation 

In [13]:
#code block 13
#merge df_movies and df_genres_pivot together

df_movies_genres = pd.merge(df_movies, df_genres_pivot, how='left', on='id')

In [14]:
#code block 14
#filters the animation column to keep movies where the column has 1

df_movies_animation = df_movies_genres[df_movies_genres['Animation'] == 1]

In [15]:
#code block 15
#keep the selected columns only from df_movies_animation

df_movies_animation = df_movies_animation[['id', 'title']]                                     
df_movies_animation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1935 entries, 0 to 45437
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1935 non-null   int64 
 1   title   1934 non-null   object
dtypes: int64(1), object(1)
memory usage: 45.4+ KB


In [16]:
#code block 16
#merge the df_movies_animation and df_ratings

df_animation_rec = pd.merge(df_movies_animation, df_ratings, left_on='id', right_on='movieId', how='inner')
df_animation_rec.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119524 entries, 0 to 119523
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         119524 non-null  int64  
 1   title      119524 non-null  object 
 2   userId     119524 non-null  int64  
 3   movieId    119524 non-null  int64  
 4   rating     119524 non-null  float64
 5   timestamp  119524 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 6.4+ MB


In [17]:
#code block 17
#views random 5 rows in df_animation_rec

df_animation_rec.sample(5)

,id,title,userId,movieId,rating,timestamp
1174,2300,Space Jam,29611,2300,5.0,1030134345
58745,919,Blood: The Last Vampire,175351,919,5.0,965186493
106784,1267,Meet the Robinsons,248825,1267,4.0,1122582722
104224,1267,Meet the Robinsons,180765,1267,5.0,1071310203
60671,919,Blood: The Last Vampire,194678,919,5.0,956527170


## Create a Recommender System using PCA

### Create a pivot table named df_comedies_pivot

- To create the first recommender system, we will first run PCA to identify the number of components to include when creating our correlation matrix

In [18]:
#code block 18
#creates a pivot table with df_animation_pivot

df_animation_pivot = pd.pivot_table(df_animation_rec, index='title', columns='userId', values='rating')
df_animation_pivot.head()

userId,8,12,15,20,27,29,36,37,43,47,49,53,54,56,62,63,65,68,70,74,76,79,89,91,96,98,120,123,125,129,132,138,139,142,144,150,151,153,160,163,166,167,169,170,174,176,191,193,201,210,212,213,217,218,221,229,230,234,235,244,246,251,265,267,271,272,277,294,296,302,303,308,316,319,321,332,337,340,342,346,348,352,353,354,359,362,368,380,382,384,389,390,391,392,398,400,404,408,410,415,423,426,427,429,431,432,434,442,443,445,447,450,451,453,461,462,463,465,468,469,471,474,475,483,491,493,499,506,512,518,521,523,527,530,532,533,540,543,545,557,561,567,573,580,581,583,584,598,600,602,604,606,607,611,614,619,624,625,627,638,642,654,667,672,674,681,682,686,688,694,698,701,702,708,711,713,719,720,724,728,738,740,741,744,755,758,764,765,766,767,773,775,783,788,790,792,799,800,805,806,807,813,823,824,825,829,830,832,834,835,836,837,839,844,849,850,851,852,856,860,865,871,872,875,877,878,882,887,896,898,908,917,921,931,936,937,949,950,959,962,963,971,988,992,993,1003,1010,1011,1012,1014,...,269837,269842,269843,269849,269854,269857,269866,269874,269884,269886,269893,269896,269899,269900,269901,269913,269916,269918,269924,269925,269935,269939,269940,269943,269948,269950,269955,269957,269960,269963,269968,269969,269974,269982,269994,269997,270012,270013,270020,270027,270031,270035,270045,270050,270054,270055,270056,270070,270071,270073,270077,270084,270085,270087,270088,270089,270093,270094,270095,270096,270097,270100,270112,270113,270115,270121,270122,270123,270126,270129,270131,270136,270137,270141,270147,270148,270156,270157,270158,270159,270160,270162,270172,270173,270177,270181,270183,270185,270186,270187,270191,270193,270194,270195,270206,270208,270209,270212,270213,270231,270235,270236,270237,270241,270249,270255,270259,270260,270265,270268,270269,270275,270277,270278,270283,270289,270291,270307,270308,270313,270316,270326,270327,270331,270336,270359,270375,270376,270377,270385,270391,270393,270397,270402,270406,270407,270408,270409,270414,270417,270418,270420,270422,270440,270441,270444,270455,270467,270470,270475,270476,270477,270478,270485,270488,270498,270505,270509,270512,270513,270514,270516,270524,270525,270531,270538,270539,270547,270548,270549,270550,270551,270555,270558,270564,270565,270579,270585,270591,270596,270609,270615,270616,270620,270622,270631,270632,270634,270636,270639,270643,270654,270655,270658,270684,270688,270690,270694,270697,270704,270706,270707,270708,270709,270721,270727,270729,270731,270734,270736,270739,270740,270741,270746,270750,270757,270759,270762,270766,270769,270784,270788,270791,270793,270795,270799,270811,270813,270817,270827,270828,270831,270835,270836,270845,270846,270847,270850,270854,270859,270871,270874,270877,270879,270880,270885,270887,270890,270893,270896
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A Chairy Tale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [19]:
#code block 19
#fills the null values with 0

df_animation_pivot = df_animation_pivot.fillna(0)
df_animation_pivot.head()

userId,8,12,15,20,27,29,36,37,43,47,49,53,54,56,62,63,65,68,70,74,76,79,89,91,96,98,120,123,125,129,132,138,139,142,144,150,151,153,160,163,166,167,169,170,174,176,191,193,201,210,212,213,217,218,221,229,230,234,235,244,246,251,265,267,271,272,277,294,296,302,303,308,316,319,321,332,337,340,342,346,348,352,353,354,359,362,368,380,382,384,389,390,391,392,398,400,404,408,410,415,423,426,427,429,431,432,434,442,443,445,447,450,451,453,461,462,463,465,468,469,471,474,475,483,491,493,499,506,512,518,521,523,527,530,532,533,540,543,545,557,561,567,573,580,581,583,584,598,600,602,604,606,607,611,614,619,624,625,627,638,642,654,667,672,674,681,682,686,688,694,698,701,702,708,711,713,719,720,724,728,738,740,741,744,755,758,764,765,766,767,773,775,783,788,790,792,799,800,805,806,807,813,823,824,825,829,830,832,834,835,836,837,839,844,849,850,851,852,856,860,865,871,872,875,877,878,882,887,896,898,908,917,921,931,936,937,949,950,959,962,963,971,988,992,993,1003,1010,1011,1012,1014,...,269837,269842,269843,269849,269854,269857,269866,269874,269884,269886,269893,269896,269899,269900,269901,269913,269916,269918,269924,269925,269935,269939,269940,269943,269948,269950,269955,269957,269960,269963,269968,269969,269974,269982,269994,269997,270012,270013,270020,270027,270031,270035,270045,270050,270054,270055,270056,270070,270071,270073,270077,270084,270085,270087,270088,270089,270093,270094,270095,270096,270097,270100,270112,270113,270115,270121,270122,270123,270126,270129,270131,270136,270137,270141,270147,270148,270156,270157,270158,270159,270160,270162,270172,270173,270177,270181,270183,270185,270186,270187,270191,270193,270194,270195,270206,270208,270209,270212,270213,270231,270235,270236,270237,270241,270249,270255,270259,270260,270265,270268,270269,270275,270277,270278,270283,270289,270291,270307,270308,270313,270316,270326,270327,270331,270336,270359,270375,270376,270377,270385,270391,270393,270397,270402,270406,270407,270408,270409,270414,270417,270418,270420,270422,270440,270441,270444,270455,270467,270470,270475,270476,270477,270478,270485,270488,270498,270505,270509,270512,270513,270514,270516,270524,270525,270531,270538,270539,270547,270548,270549,270550,270551,270555,270558,270564,270565,270579,270585,270591,270596,270609,270615,270616,270620,270622,270631,270632,270634,270636,270639,270643,270654,270655,270658,270684,270688,270690,270694,270697,270704,270706,270707,270708,270709,270721,270727,270729,270731,270734,270736,270739,270740,270741,270746,270750,270757,270759,270762,270766,270769,270784,270788,270791,270793,270795,270799,270811,270813,270817,270827,270828,270831,270835,270836,270845,270846,270847,270850,270854,270859,270871,270874,270877,270879,270880,270885,270887,270890,270893,270896
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A Chairy Tale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [20]:
#code block 20
#creates a variable for list of column names

animation_names = df_animation_pivot.index
animation_names

Index(['A Chairy Tale', 'A Close Shave', 'A Grand Day Out', 'Aladdin',
       'Alvin and the Chipmunks', 'Antz', 'Asterix vs. Caesar',
       'Bambi Meets Godzilla', 'Beavis and Butt-Head Do America', 'Bee Movie',
       'Blood: The Last Vampire',
       'Bon Voyage, Charlie Brown (and Don't Come Back!)',
       'Bugs Bunny's 3rd Movie: 1001 Rabbit Tales',
       'Cardcaptor Sakura: The Sealed Card', 'Cars', 'Cars 2', 'Chicken Run',
       'Coonskin', 'Dibu 2: La venganza de Nasty', 'Dibu: La Película',
       'Fallen Art', 'Feed the Kitty', 'Food', 'Foodfight!', 'Frank & Wendy',
       'Futurama: Bender's Big Score', 'Garfield', 'Garfield's Fun Fest',
       'Gertie the Dinosaur', 'Gestalt', 'Happily N'Ever After',
       'How to Hook Up Your Home Theater', 'Hundhotellet', 'Ice Age',
       'Ice Age: The Meltdown', 'Inside Out', 'Jackboots on Whitehall',
       'Lissi and the Wild Emperor', 'Lotte from Gadgetville',
       'Love Hina Spring Special - I Wish Your Dream', 'Madly Madagas

### Create a PCA for df_comedy_pivot 

- Time to reduce the number of dimensions from 64,272 (number of columns in df_animation_pivot) to a more reasonable number that still captures the explained variance

In [21]:
#code block 21

from sklearn.decomposition import PCA

In [22]:
#code block 22

# Set number of principal components for model
n_pcs = 11
model = PCA(n_components=n_pcs)

# Fit the model - return projected data
df_animation_pca = model.fit_transform(df_animation_pivot)

#Create a DataFrame of the projected data
df_animation_pca = pd.DataFrame(df_animation_pca)
df_animation_pca.head()

,0,1,2,3,4,5,6,7,8,9,10
0,-27.130750,-2.966546,-7.587453,-8.345939,-3.178857,-3.336733,-3.804461,-1.791554,-1.526378,-2.143334,-2.220910
1,26.971490,-6.578131,11.365766,79.046633,9.490795,113.396308,23.291428,197.776366,1.650802,-19.385908,-43.077725
2,-26.451624,-2.821939,-7.201223,-8.140370,-3.002843,-3.229293,-3.455761,-1.520686,-1.317367,-1.974931,-2.146192
3,-26.944526,-2.954584,-7.389997,-8.159210,-3.244301,-3.154702,-3.613611,-1.524250,-1.530369,-2.122547,-2.231520
4,-22.146761,-1.332476,-5.419029,-7.672488,-1.771934,-3.369632,-2.708785,-2.270637,0.603160,-2.561983,-2.725248


In [23]:
#code block 23

names = []
for x in range(n_pcs):
    names.append('PC%d'%(x+1))

df_animation_pca.columns = names
df_animation_pca.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11
0,-27.130750,-2.966546,-7.587453,-8.345939,-3.178857,-3.336733,-3.804461,-1.791554,-1.526378,-2.143334,-2.220910
1,26.971490,-6.578131,11.365766,79.046633,9.490795,113.396308,23.291428,197.776366,1.650802,-19.385908,-43.077725
2,-26.451624,-2.821939,-7.201223,-8.140370,-3.002843,-3.229293,-3.455761,-1.520686,-1.317367,-1.974931,-2.146192
3,-26.944526,-2.954584,-7.389997,-8.159210,-3.244301,-3.154702,-3.613611,-1.524250,-1.530369,-2.122547,-2.231520
4,-22.146761,-1.332476,-5.419029,-7.672488,-1.771934,-3.369632,-2.708785,-2.270637,0.603160,-2.561983,-2.725248


In [24]:
#code block 24

explained_ratio = model.explained_variance_ratio_
explained_ratio = pd.DataFrame(explained_ratio)
explained_ratio = explained_ratio.rename(columns = {0: 'explained_ratio'})
explained_ratio['explained_ratio_sum']= np.cumsum(explained_ratio.explained_ratio)
explained_ratio

,explained_ratio,explained_ratio_sum
0,0.343131,0.343131
1,0.117345,0.460477
2,0.099515,0.559991
3,0.073642,0.633633
4,0.059307,0.692940
5,0.052614,0.745554
6,0.045187,0.790741
7,0.033803,0.824544
8,0.031018,0.855562
9,0.027427,0.882989


### Create a Correlation Matrix for a Recommender System

In [25]:
#code block 25
#creates a correlation matric using df_animation_pca

corr_mat_animation = np.corrcoef(df_animation_pca)
corr_mat_animation.shape

(72, 72)

In [26]:
#code block 26

corr_mat_animation

array([[1.        , 0.02304253, 0.999931  , ..., 0.99990566, 0.9998865 ,
        0.999999  ],
       [0.02304253, 1.        , 0.02513601, ..., 0.02231739, 0.0285814 ,
        0.02287037],
       [0.999931  , 0.02513601, 1.        , ..., 0.99986464, 0.99981661,
        0.9999207 ],
       ...,
       [0.99990566, 0.02231739, 0.99986464, ..., 1.        , 0.99974192,
        0.99990225],
       [0.9998865 , 0.0285814 , 0.99981661, ..., 0.99974192, 1.        ,
        0.99988024],
       [0.999999  , 0.02287037, 0.9999207 , ..., 0.99990225, 0.99988024,
        1.        ]])

In [27]:
#code block 27

corr_mat_animation=pd.DataFrame(corr_mat_animation,columns=(animation_names))
#sets the index=animation_names
corr_mat_animation.index=animation_names 
corr_mat_animation

title,A Chairy Tale,A Close Shave,A Grand Day Out,Aladdin,Alvin and the Chipmunks,Antz,Asterix vs. Caesar,Bambi Meets Godzilla,Beavis and Butt-Head Do America,Bee Movie,Blood: The Last Vampire,"Bon Voyage, Charlie Brown (and Don't Come Back!)",Bugs Bunny's 3rd Movie: 1001 Rabbit Tales,Cardcaptor Sakura: The Sealed Card,Cars,Cars 2,Chicken Run,Coonskin,Dibu 2: La venganza de Nasty,Dibu: La Película,Fallen Art,Feed the Kitty,Food,Foodfight!,Frank & Wendy,Futurama: Bender's Big Score,Garfield,Garfield's Fun Fest,Gertie the Dinosaur,Gestalt,Happily N'Ever After,How to Hook Up Your Home Theater,Hundhotellet,Ice Age,Ice Age: The Meltdown,Inside Out,Jackboots on Whitehall,Lissi and the Wild Emperor,Lotte from Gadgetville,Love Hina Spring Special - I Wish Your Dream,Madly Madagascar,Meet the Robinsons,"Monsters, Inc.",Mr. Bug Goes to Town,Mutant Aliens,Ratatouille,Scary Godmother: Halloween Spooktakular,Scooby-Doo! and the Reluctant Werewolf,SeaFood,Shrek,Shrek 2,Shrek the Third,Slayers Premium,Slayers Return,Space Jam,Superstar Goofy,TMNT,Team America: World Police,The Boxtrolls,The Cat Came Back,The Curse of the Were-Rabbit,The Prince and the Pauper,The Simpsons Movie,The Smurfs 2,The Substitute,The Wind in the Willows,The Wrong Trousers,Tom and Jerry: Shiver Me Whiskers,Toy Story,Urusei Yatsura 2: Beautiful Dreamer,Who Framed Roger Rabbit,"Why, Charlie Brown, Why?"
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A Chairy Tale,1.000000,0.023043,0.999931,0.999894,0.987822,0.945140,0.999253,0.980989,0.611001,0.999894,-0.865559,0.999984,0.999941,0.993916,-0.533917,0.996138,0.999865,0.991653,0.999808,0.999999,0.999606,0.999766,0.999955,0.992278,0.999994,0.998550,0.989325,0.999745,0.999998,0.999985,0.999580,0.999892,0.999985,0.862455,-0.089151,0.999998,0.999993,0.998500,-0.264017,0.999979,0.999990,-0.540912,-0.188528,0.964003,0.999975,0.991934,0.999896,0.999148,0.999993,0.937238,0.755561,0.886365,0.999999,0.999997,-0.305870,0.999990,0.109398,0.899169,1.000000,0.999733,0.172690,0.999598,0.661337,0.999890,0.999993,0.999938,-0.304392,0.986852,0.987761,0.999906,0.999886,0.999999
A Close Shave,0.023043,1.000000,0.025136,0.034462,-0.037354,-0.023272,0.012634,-0.044515,0.299590,0.029860,0.037811,0.023849,0.024098,-0.025877,-0.151717,0.030943,0.017175,-0.015422,0.027089,0.022570,0.015318,0.014091,0.017100,0.041706,0.021332,0.020465,-0.015780,0.018721,0.021863,0.020095,0.016031,0.032675,0.023336,0.187973,-0.310484,0.024596,0.023854,0.036777,-0.143706,0.023892,0.023435,-0.068092,0.087904,-0.078877,0.020597,0.036313,0.017993,0.003726,0.021950,0.065297,0.096211,0.137887,0.022597,0.022535,-0.163889,0.022429,-0.250405,0.017116,0.022893,0.014826,-0.071115,0.012480,0.363413,0.018318,0.025026,0.016939,0.026499,-0.001070,0.097355,0.022317,0.028581,0.022870
A Grand Day Out,0.999931,0.025136,1.000000,0.999898,0.988432,0.942824,0.999073,0.982287,0.609460,0.999868,-0.864803,0.999928,0.999920,0.994756,-0.536350,0.995458,0.999906,0.992084,0.999814,0.999933,0.999723,0.999830,0.999952,0.992054,0.999943,0.998663,0.990069,0.999823,0.999923,0.999946,0.999687,0.999866,0.999922,0.863423,-0.084305,0.999936,0.999934,0.998128,-0.257576,0.999939,0.999910,-0.538508,-0.195534,0.965723,0.999920,0.991727,0.999906,0.999348,0.999923,0.934715,0.751438,0.882946,0.999934,0.999936,-0.306217,0.999955,0.111552,0.902131,0.999929,0.999776,0.167283,0.999736,0.661640,0.999825,0.999945,0.999918,-0.306712,0.987239,0.988329,0.999865,0.999817,0.999921
Aladdin,0.999894,0.034462,0.999898,1.000000,0.986850,0.943034,0.998997,0.980352,0.610780,0.999843,-0.866201,0.999896,0.999895,0.993642,-0.538309,0.995794,0.999774,0.991420,0.999824,0.999893,0.999514,0.999666,0.999805,0.992219,0.999878,0.998299,0.988798,0.999751,0.999883,0.999869,0.999396,0.999896,0.999896,0.864191,-0.092813,0.999916,0.999906,0.998447,-0.259871,0.999906,0.999903,-0.537344,-0.187675,0.964155,0.999852,0.991533,0.999803,0.998912,0.999867,0.937128,0.755955,0.886936,0.999893,0.999891,-0.311781,0.999899,0

In [28]:
#code block 28
#creates a dataframe that includes only Antz column

corr_Antz = corr_mat_animation[['Antz']]

In [29]:
#code block 29
#filters the dataframe to keep movies with less than 0.9 correlation so as not to show Antz as part of the movie
#shows the top 10 movies that are highly correlated to Antz

corr_Antz = corr_Antz[corr_Antz['Antz']<.9999]
corr_Antz = corr_Antz.sort_values(ascending=False, by='Antz').head(10)
corr_Antz

title,Antz
title,
Ratatouille,0.961529
Shrek,0.954367
Asterix vs. Caesar,0.954239
Cars 2,0.951485
Lissi and the Wild Emperor,0.947782
Who Framed Roger Rabbit,0.946741
Bee Movie,0.946650
SeaFood,0.945514
"Why, Charlie Brown, Why?",0.945396


### Explain the results for the Recommender System for Antz

The animated movie choices highly correlated to Antz are seen above with Ratatouille - which was one of my favorite animations to watch back then - being the most correlated, that is, the immediate animation movie to see after Antz. Then Shrek and some other animated movies in the table.  

## Create a Recommender System using SVD 

### Create a correlation matrix from SVD

- Considering using SVD as a better recommender system because it does better with sparse data

In [30]:
#code block 30

from sklearn.decomposition import TruncatedSVD

In [31]:
#code block 31

SVD = TruncatedSVD(n_components=11, random_state=42)
resultant_matrix_SVD = SVD.fit_transform(df_animation_pivot)
resultant_matrix_SVD.shape

(72, 11)

In [32]:
#code block 32
#creates a correlation matrix from SVD

corr_mat_SVD_animation = np.corrcoef(resultant_matrix_SVD)
corr_mat_SVD_animation

array([[ 1.        ,  0.18528558,  0.62400191, ...,  0.50316768,
         0.31331844,  0.69745277],
       [ 0.18528558,  1.        ,  0.1706159 , ..., -0.03728835,
         0.31122897,  0.05734326],
       [ 0.62400191,  0.1706159 ,  1.        , ...,  0.28464678,
         0.36020079,  0.44271402],
       ...,
       [ 0.50316768, -0.03728835,  0.28464678, ...,  1.        ,
        -0.03513663,  0.29698749],
       [ 0.31331844,  0.31122897,  0.36020079, ..., -0.03513663,
         1.        ,  0.13110174],
       [ 0.69745277,  0.05734326,  0.44271402, ...,  0.29698749,
         0.13110174,  1.        ]])

In [33]:
#code block 33

corr_mat_SVD_animation = pd.DataFrame(corr_mat_SVD_animation, columns=(animation_names))
corr_mat_SVD_animation.index = animation_names
corr_mat_SVD_animation

title,A Chairy Tale,A Close Shave,A Grand Day Out,Aladdin,Alvin and the Chipmunks,Antz,Asterix vs. Caesar,Bambi Meets Godzilla,Beavis and Butt-Head Do America,Bee Movie,Blood: The Last Vampire,"Bon Voyage, Charlie Brown (and Don't Come Back!)",Bugs Bunny's 3rd Movie: 1001 Rabbit Tales,Cardcaptor Sakura: The Sealed Card,Cars,Cars 2,Chicken Run,Coonskin,Dibu 2: La venganza de Nasty,Dibu: La Película,Fallen Art,Feed the Kitty,Food,Foodfight!,Frank & Wendy,Futurama: Bender's Big Score,Garfield,Garfield's Fun Fest,Gertie the Dinosaur,Gestalt,Happily N'Ever After,How to Hook Up Your Home Theater,Hundhotellet,Ice Age,Ice Age: The Meltdown,Inside Out,Jackboots on Whitehall,Lissi and the Wild Emperor,Lotte from Gadgetville,Love Hina Spring Special - I Wish Your Dream,Madly Madagascar,Meet the Robinsons,"Monsters, Inc.",Mr. Bug Goes to Town,Mutant Aliens,Ratatouille,Scary Godmother: Halloween Spooktakular,Scooby-Doo! and the Reluctant Werewolf,SeaFood,Shrek,Shrek 2,Shrek the Third,Slayers Premium,Slayers Return,Space Jam,Superstar Goofy,TMNT,Team America: World Police,The Boxtrolls,The Cat Came Back,The Curse of the Were-Rabbit,The Prince and the Pauper,The Simpsons Movie,The Smurfs 2,The Substitute,The Wind in the Willows,The Wrong Trousers,Tom and Jerry: Shiver Me Whiskers,Toy Story,Urusei Yatsura 2: Beautiful Dreamer,Who Framed Roger Rabbit,"Why, Charlie Brown, Why?"
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A Chairy Tale,1.000000,0.185286,0.624002,0.332394,0.353251,0.477902,0.308638,0.191394,0.530273,0.469229,0.907250,0.232174,0.549302,0.262481,0.101264,0.754692,0.186524,0.286919,0.464308,0.263490,0.366951,0.202550,0.401958,0.833964,-0.038636,0.429448,0.246634,0.368305,0.233820,0.361204,0.276822,0.717893,0.429031,0.418334,0.206146,0.335125,0.541194,0.709745,0.116177,0.057779,-0.129890,0.064421,0.102733,0.234206,0.695833,0.463149,0.315193,0.545948,0.493834,0.490902,0.434401,0.559279,0.408059,0.311826,0.116524,0.354904,0.198582,0.401545,0.999990,0.322541,0.188557,0.376349,0.372857,0.582610,0.441617,0.417785,0.050288,0.634298,0.490635,0.503168,0.313318,0.697453
A Close Shave,0.185286,1.000000,0.170616,0.730689,-0.112388,0.019198,-0.042563,-0.142536,0.190714,0.325590,0.144357,0.232441,0.128722,-0.272057,-0.048810,0.124158,-0.228377,-0.270352,0.225271,-0.206618,-0.228532,-0.327915,-0.397915,0.143085,-0.495950,0.078598,-0.089410,-0.026847,-0.525513,-0.453935,-0.090865,0.575724,0.115203,0.244476,-0.257583,0.816404,0.255582,0.240335,-0.087478,0.209899,0.213799,0.034654,0.123288,-0.299489,-0.233643,0.140448,-0.137059,-0.154875,-0.145947,0.128627,0.116938,0.236465,-0.220155,-0.041791,-0.080958,-0.047319,-0.254861,0.087357,0.185612,-0.323769,-0.083489,-0.307574,0.383999,-0.283133,0.576095,-0.274348,0.082361,-0.013999,0.356815,-0.037288,0.311229,0.057343
A Grand Day Out,0.624002,0.170616,1.000000,0.596522,0.802973,0.689130,0.724362,0.797635,0.805967,0.846973,0.816213,0.564478,0.602079,0.771387,0.367497,0.679784,0.705857,0.365911,0.785956,0.538301,0.528286,0.582454,0.786835,0.738634,0.066226,0.793848,0.752558,0.868851,0.204122,0.441700,0.765497,0.616442,0.686335,0.829774,0.339486,0.471133,0.761753,0.591370,0.552741,0.547543,0.472509,0.637729,0.009149,0.596976,0.515563,0.809707,0.812150,0.886047,0.458317,0.453646,0.135919,0.379191,0.505174,0.796639,0.333526,0.640373,0.130826,0.874484,0.622272,0.393769,-0.109431,0.589024,0.640320,0.261604,0.501893,0.770144,0.327191,0.799989,0.820576,0.284647,0.360201,0.442714
Aladdin,0.332394,0.730689,0.596522,1.000000,0.168367,0.243726,0.277110,0.191904,0.401075,0.522773,0.367490,0.450580,0.539074,0.199616,-0.043628,0.346061,0.306232,0.073308,0.607042,0.279971,0.199317,0.214003,0.101097,0.421418,-0.244813,0.285038,0.183748,0.559229,-0.131767,0.036703,0.189614,0.655229,0.486155,0.492888,-0.130632,0.828963,0.568236,0.421608,0.396292,0.492930,0.530577,0.468667,0.184939,0.242454,0.166396,0.366949,0.392439,0.287122,-0.015921,0.337987,0.232118,0.384062,0.244390,0.391576,-0.166224,0.4

In [34]:
#code block 34
#creates a dataframe that includes only Antz column 

corr_SVD_Antz = corr_mat_SVD_animation[['Antz']]

In [35]:
#code block 35
#filters the dataframe to keep movies with less than 0.9 correlation so as not to show Antz as part of the movie
#shows the top 10 movies that are highly correlated to Antz

corr_SVD_Antz = corr_SVD_Antz[corr_SVD_Antz['Antz']<.9999]
corr_SVD_Antz = corr_SVD_Antz.sort_values(ascending=False, by='Antz').head(10)
corr_SVD_Antz

title,Antz
title,
Asterix vs. Caesar,0.946045
Ratatouille,0.919399
Beavis and Butt-Head Do America,0.912824
Cars 2,0.862850
Bee Movie,0.846793
Shrek,0.807742
Ice Age,0.802760
Lissi and the Wild Emperor,0.771770
Blood: The Last Vampire,0.752055


### Explain the results for the Recommender System using SVD for Antz

The choices of the next 10 movies to watch after seeing Antz with SVD in the recommender system seem quite surprising to me. Seeing 'Blood: The Last Vampire' on this list surprises me, considering it is a horror animated movie. It is important to note that animated movies such as 'Beavis and Butt-Head Do America,' 'Blood: The Last Vampire,' and 'Foodfight!' were not included in the PCA recommender system. Additionally, the order of arrangement is not the same for the movies that were in the other recommender system; while Ratatouille topped the PCA system, 'Asterix vs. Caesar' tops the SVD system of animated movies to watch next after Antz.